## 一、读取数据

In [1]:
import pandas as pd

df = pd.read_csv('data/知乎-全职儿女.csv', encoding='utf-8')
df.dropna(subset=['content'], inplace=True)
print('记录数: ', len(df))
df.head(2)

记录数:  411


name                            userID  gender  \
0   曙光  2f4073a75fd09e7bff6b4fc0f863d788       1   
1  林先生  88c229633790df7350f1bfb9f0282145       1   

                        headline  is_advertiser  is_org   utype  can_comment  \
0   共产主义万岁！我愿意将自己的一腔热血奉献给这个美好的社会          False   False  people         True   
1  我们应该神人般思考，却需要凡人般生活。彼岸理想不可此岸化。          False   False  people         True   

   follower_count                                            content  \
0             155  首先，我认为「全职儿女」不应该被简单地归为啃老。在目前社会环境下，随着经济、教育等发展，年轻...   
1          272953  这时候想起“个人选择”了。 首先，他是不是一个个人选择，他是一个个人选择。所以作为个人选择，...   

   updated_time  voteup_count  
0    1682334790             0  
1    1682334654             6

<br><br>

## 二、清洗数据

In [2]:
import re
import jieba

stoptext = open('data/stopwords.txt', encoding='utf-8').read()        
stopwords = stoptext.split('\n')  


def clean_text(text):
    # 用正则表达式提取中文文本
    text = ''.join(re.findall('[\u4e00-\u9fa5]+', text))   
    words = jieba.lcut(text)                               
    words = [w for w in words if w not in stopwords]   
    #整理成用空格间隔词语的文本形式(类似西方语言)
    return ' '.join(words)


test_text = "首先，我认为「全职儿女」不应该被简单地归为啃老。在目前社会环境下，随着经济、教育等发展，年轻..."
clean_text(text=test_text)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/y0/4gqxky0s2t94x1c1qhlwr6100000gn/T/jieba.cache
Loading model cost 0.280 seconds.
Prefix dict has been built successfully.


'全职 儿女 简单 地归为 啃 老 社会 环境 经济 教育 发展 年轻'

In [3]:
df['clean_content'] = df['content'].apply(clean_text)
df.head(1)

name                            userID  gender  \
0   曙光  2f4073a75fd09e7bff6b4fc0f863d788       1   

                       headline  is_advertiser  is_org   utype  can_comment  \
0  共产主义万岁！我愿意将自己的一腔热血奉献给这个美好的社会          False   False  people         True   

   follower_count                                            content  \
0             155  首先，我认为「全职儿女」不应该被简单地归为啃老。在目前社会环境下，随着经济、教育等发展，年轻...   

   updated_time  voteup_count  \
0    1682334790             0   

                                       clean_content  
0  全职 儿女 简单 地归为 啃 老 社会 环境 经济 教育 发展 年轻人 处于 灵活 就业 状...

<br><br>

## 三、训练LDA模型

### 3.1 训练
![](img/03-wordcloud.png)

根据词云图， 假设对数据比较了解，可以直接设置话题数 n_components = 4


In [4]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 构建词典，将词转为数字。将文档转为向量
vectorizer = TfidfVectorizer(max_df=0.5, min_df=20)   
doc_term_matrix = vectorizer.fit_transform(df['clean_content'])

# 构建LDA话题模型
# 初始化模型，设置话题数为4,随机状态码888
lda_model = LatentDirichletAllocation(n_components=4, random_state=888)       
lda_output = lda_model.fit_transform(doc_term_matrix)
lda_model

LatentDirichletAllocation(n_components=4, random_state=888)

<br>

### 3.2 保存模型
如果训练过程非常久，保存模型，下次就可以跳过训练阶段，直接使用模型。

In [5]:
import joblib
# # 保存模型
joblib.dump(lda_model, 'output/全职儿女lda_model.pkl')

['output/全职儿女lda_model.pkl']

<br>

### 3.3 导入模型

In [6]:
import joblib

"""导入保存的模型"""
lda_model = joblib.load('output/全职儿女lda_model.pkl')
lda_model

LatentDirichletAllocation(n_components=4, random_state=888)

<br><br>

## 四、使用LDA模型
### 4.1 查看话题特征词
获得每个话题对应的的n个特征词，方便后续对每个话题命名和解读

In [16]:
import numpy as np


def show_topics(vectorizer, lda_model, top_n=30):
    """
    显示每个话题最重要的n个词语
    vectorizer: 词袋法或tfidf.基于前面代码这里使用TF-IDF法
    lda_model: 训练好的lda话题模型
    top_n: 设置最重要的n个特征词，默认30个.
    """
    
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:top_n]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords


"""利用show_topics函数展示全职儿女文本中的4个话题，基于每个话题最重要的20个词语为每个话题命名"""
topic_keywords = show_topics(vectorizer=vectorizer,   # 【可改动】vectorizer我们训练的词语空间
                             lda_model=lda_model,     # 【可改动】lda_model训练的lda模型
                             top_n=20)                # 【可改动】最重要的30个词语

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word-' + str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic-' + str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

Word-0 Word-1 Word-2 Word-3 Word-4 Word-5 Word-6 Word-7 Word-8 Word-9  \
Topic-0     选择     一个    年轻人     家庭     经济     职业     就业     灵活     生活     社会   
Topic-1     工作     家里     在家     父母     照顾     社会     陪伴     工资     能力    年轻人   
Topic-2     媒体     不到     一种     工作     毕业     社会     一个     很多     灵活     就业   
Topic-3     父母     图片     子女     孩子     老人     就业     家务     很多     收入     照顾   

        Word-10 Word-11 Word-12 Word-13 Word-14 Word-15 Word-16 Word-17  \
Topic-0      方式      一种      毕业      父母      支持      情况      偏见      能力   
Topic-1      生活      一个      收入      家庭      毕业      支持      老人      职业   
Topic-2      在家      收入      子女      生活      照顾      父母      家庭      情况   
Topic-3      偏见      家庭      情况      家里      在家      一个      生活      一种   

        Word-18 Word-19  
Topic-0      照顾      很多  
Topic-1      一种      孩子  
Topic-2      家务      孩子  
Topic-3      职业      经济


<br>

### 4.2 预测文本的话题id

In [10]:
import numpy as np

def predict_topic(text):
    doc_term_matrix = vectorizer.transform([clean_text(text)])
    topic_term_prob_matrix = lda_model.transform(doc_term_matrix)
    topic_index = np.argmax(topic_term_prob_matrix)

    return topic_index

test_text2 = "最近，“全职儿女”话题受到舆论关注。“全职儿女”是指一种新型的脱产生活方式，年轻人脱产寄居父母生活，并通过付出一定的劳动换取经济支持，同时保持学习提升或发展副业的状态。这种生活方式既有其合理性和正当性，也有其问题和风险。我们不能一概而论，也不能一味否定或肯定。"

topic_index = predict_topic(test_text2)
print("该文本所属Topic: ", topic_index)

该文本所属Topic:  0


In [13]:
#批量预测
df['话题ID'] = df['clean_content'].apply(predict_topic)
df.head(1)

name                            userID  gender  \
0   曙光  2f4073a75fd09e7bff6b4fc0f863d788       1   

                       headline  is_advertiser  is_org   utype  can_comment  \
0  共产主义万岁！我愿意将自己的一腔热血奉献给这个美好的社会          False   False  people         True   

   follower_count                                            content  \
0             155  首先，我认为「全职儿女」不应该被简单地归为啃老。在目前社会环境下，随着经济、教育等发展，年轻...   

   updated_time  voteup_count  \
0    1682334790             0   

                                       clean_content  话题ID  
0  全职 儿女 简单 地归为 啃 老 社会 环境 经济 教育 发展 年轻人 处于 灵活 就业 状...     0

In [15]:
df['话题ID'].value_counts()

话题ID
0    214
1     88
3     84
2     25
Name: count, dtype: int64

In [17]:
#预测结果保存到csv、xlsx中。
df.to_csv('output/话题预测结果.csv', index=False)
df.to_excel('output/话题预测结果.xlsx', index=False)